In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sklearn
sklearn.__version__

'1.0.1'

In [3]:
#import comet_ml in the top of your file
from comet_ml import Experiment
import os
import matplotlib.pyplot as plt
import joblib
from copy import deepcopy
import numpy as np
from sklearn.feature_selection import RFECV
import json
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from sklearn import feature_selection
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import xgboost as xgb
import pandas as pd
from ift6758.features.feature_engineering2 import SeasonDataSetTwo
from ift6758.metrics import plot_metrics
import random
random_state = 42
import tempfile


In [4]:
years = [2019]
dataset = SeasonDataSetTwo(years)
df_total = dataset.get_features_2()
df = df_total


File already Exists, loading from ../ift6758/data/PICKLE//2019_features2.pkl


In [5]:
X = df[["shotDistance","shotAngle"]].iloc[:5]
X

,shotDistance,shotAngle
0,4.123106,14.036243
1,57.035077,-2.009554
2,26.683328,12.994617
3,36.055513,-33.690068
4,55.226805,-31.675469


In [7]:
import requests
import json
r = requests.post(
	"http://127.0.0.1:5000/predict", 
	json=X.to_json(orient="table")
)
print(r.json())



{"probaIsGoal":{"0":0.1542563787,"1":0.0567860263,"2":0.1020496683,"3":0.0856855542,"4":0.0589828114},"predictionIsGoal":{"0":0,"1":0,"2":0,"3":0,"4":0}}


In [8]:
import json
import requests
import pandas as pd
from ift6758.ift6758.client.serving_client import ServingClient

In [9]:
sc = ServingClient()


In [10]:
sc.predict(X)

,probaIsGoal,predictionIsGoal
0,0.154256,0
1,0.056786,0
2,0.102050,0
3,0.085686,0
4,0.058983,0


In [11]:
sc.logs()

"INFO:app:Welcome to the work of Team 11\nINFO:app:Initialization\nINFO:app:Default Classifier Loaded logreg-distance-angle\nINFO:app:{\"schema\":{\"fields\":[{\"name\":\"index\",\"type\":\"integer\"},{\"name\":\"shotDistance\",\"type\":\"number\"},{\"name\":\"shotAngle\",\"type\":\"number\"}],\"primaryKey\":[\"index\"],\"pandas_version\":\"0.20.0\"},\"data\":[{\"index\":0,\"shotDistance\":4.1231056256,\"shotAngle\":14.0362434679},{\"index\":1,\"shotDistance\":57.0350769264,\"shotAngle\":-2.009553813},{\"index\":2,\"shotDistance\":26.6833281283,\"shotAngle\":12.9946167919},{\"index\":3,\"shotDistance\":36.0555127546,\"shotAngle\":-33.690067526},{\"index\":4,\"shotDistance\":55.2268050859,\"shotAngle\":-31.6754687381}]}\nINFO:app:{\"probaIsGoal\":{\"0\":0.1542563787,\"1\":0.0567860263,\"2\":0.1020496683,\"3\":0.0856855542,\"4\":0.0589828114},\"predictionIsGoal\":{\"0\":0,\"1\":0,\"2\":0,\"3\":0,\"4\":0}}\nINFO:werkzeug:127.0.0.1 - - [18/Dec/2021 15:23:54] \"POST /predict HTTP/1.1\" 200 

'"INFO:app:Welcome to the work of Team 11\\nINFO:app:Initialization\\nINFO:app:Default Classifier Loaded logreg-distance-angle\\nINFO:app:{\\"schema\\":{\\"fields\\":[{\\"name\\":\\"index\\",\\"type\\":\\"integer\\"},{\\"name\\":\\"shotDistance\\",\\"type\\":\\"number\\"},{\\"name\\":\\"shotAngle\\",\\"type\\":\\"number\\"}],\\"primaryKey\\":[\\"index\\"],\\"pandas_version\\":\\"0.20.0\\"},\\"data\\":[{\\"index\\":0,\\"shotDistance\\":4.1231056256,\\"shotAngle\\":14.0362434679},{\\"index\\":1,\\"shotDistance\\":57.0350769264,\\"shotAngle\\":-2.009553813},{\\"index\\":2,\\"shotDistance\\":26.6833281283,\\"shotAngle\\":12.9946167919},{\\"index\\":3,\\"shotDistance\\":36.0555127546,\\"shotAngle\\":-33.690067526},{\\"index\\":4,\\"shotDistance\\":55.2268050859,\\"shotAngle\\":-31.6754687381}]}\\nINFO:app:{\\"probaIsGoal\\":{\\"0\\":0.1542563787,\\"1\\":0.0567860263,\\"2\\":0.1020496683,\\"3\\":0.0856855542,\\"4\\":0.0589828114},\\"predictionIsGoal\\":{\\"0\\":0,\\"1\\":0,\\"2\\":0,\\"3\\":

In [12]:
r=sc.download_registry_model(workspace="morph-e",model="xgbase-bayestuning",version="1.0.0")


In [13]:
r.json()

{'new_classifier': 'xgbase-bayestuning'}